<a href="https://colab.research.google.com/github/roxacarv/sistemaderecomendacao_mal/blob/master/Dados_MAL_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sistema de Recomendação de Animes usando KNN

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Importando bibliotecas necessárias

In [2]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

##Importando dataset de animes

In [3]:
# Importando os dados relacionados aos animes
animes = pd.read_csv("drive/MyDrive/animal_mal/anime.csv", encoding="latin-1")

In [4]:
animes=animes[["MAL_ID", "Name", "Genres", "English name", "Type", "Rating"]]
animes["Type"].value_counts()

TV         4996
OVA        3894
Movie      3041
Special    2218
ONA        1907
Music      1469
Unknown      37
Name: Type, dtype: int64

## Importando dataset de perfis de usuários

In [5]:
# Importando os dados de ratings
ratings = pd.read_csv("drive/MyDrive/animal_mal/rating_complete.csv", encoding="latin-1")

In [6]:
animes["English name"] = animes["English name"].replace("Unknown", np.NaN)
animes["English name"].fillna(animes.Name, inplace=True)

In [95]:
ratings["user_id"].value_counts()

189037    15455
162615    14864
68042     13462
283786    12778
259790     9996
          ...  
201316        1
201300        1
97555         1
201293        1
137466        1
Name: user_id, Length: 310059, dtype: int64

In [7]:
x = ratings["user_id"].value_counts() > 20

In [8]:
x

189037     True
162615     True
68042      True
283786     True
259790     True
          ...  
201316    False
201300    False
97555     False
201293    False
137466    False
Name: user_id, Length: 310059, dtype: bool

In [9]:
y = x[x].index
y.shape

(263628,)

In [10]:
ratings = ratings[ratings["user_id"].isin(y)]

In [11]:
ratings

,user_id,anime_id,rating
0,0,430,9
1,0,1004,5
2,0,3010,7
3,0,570,7
4,0,2762,9
...,...,...,...
57633273,353404,502,8
57633274,353404,987,4
57633275,353404,225,8
57633276,353404,243,7


In [12]:
animes.rename(columns={"MAL_ID": "anime_id"}, inplace=True)

In [13]:
animes_ratings = ratings.merge(animes, on="anime_id")
animes_ratings.head()

,user_id,anime_id,rating,Name,Genres,English name,Type,Rating
0,0,430,9,Fullmetal Alchemist: The Conqueror of Shamballa,"Military, Comedy, Historical, Drama, Fantasy, ...",Fullmetal Alchemist:The Movie - Conqueror of S...,Movie,PG-13 - Teens 13 or older
1,6,430,8,Fullmetal Alchemist: The Conqueror of Shamballa,"Military, Comedy, Historical, Drama, Fantasy, ...",Fullmetal Alchemist:The Movie - Conqueror of S...,Movie,PG-13 - Teens 13 or older
2,18,430,10,Fullmetal Alchemist: The Conqueror of Shamballa,"Military, Comedy, Historical, Drama, Fantasy, ...",Fullmetal Alchemist:The Movie - Conqueror of S...,Movie,PG-13 - Teens 13 or older
3,19,430,8,Fullmetal Alchemist: The Conqueror of Shamballa,"Military, Comedy, Historical, Drama, Fantasy, ...",Fullmetal Alchemist:The Movie - Conqueror of S...,Movie,PG-13 - Teens 13 or older
4,33,430,4,Fullmetal Alchemist: The Conqueror of Shamballa,"Military, Comedy, Historical, Drama, Fantasy, ...",Fullmetal Alchemist:The Movie - Conqueror of S...,Movie,PG-13 - Teens 13 or older


In [14]:
animes_ratings_times = animes_ratings.groupby("anime_id")["rating"].count().reset_index()

In [15]:
animes_ratings_times

,anime_id,rating
0,1,78074
1,5,33311
2,6,46293
3,7,10143
4,8,1505
...,...,...
16867,47614,62
16868,47616,1745
16869,47618,16
16870,48177,33


In [16]:
animes_ratings_times.rename(columns={"rating": "rating_count"}, inplace=True)
animes_ratings_times

,anime_id,rating_count
0,1,78074
1,5,33311
2,6,46293
3,7,10143
4,8,1505
...,...,...
16867,47614,62
16868,47616,1745
16869,47618,16
16870,48177,33


In [1]:
f_ratings = animes_ratings.merge(animes_ratings_times, on="anime_id")
f_ratings

NameError: ignored

## Importando dataset de reviews dos usuários

In [94]:
ratings

,user_id,anime_id,rating,rating_counts
0,0,430,9,35
1,0,1004,5,35
2,0,3010,7,35
3,0,570,7,35
4,0,2762,9,35
...,...,...,...,...
57633273,353404,502,8,30
57633274,353404,987,4,30
57633275,353404,225,8,30
57633276,353404,243,7,30


# Pré-processamento de dados


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,profile,anime_uid,user_score
0,6837,34096,8
1,32404,34599,10
2,44549,28891,7
3,35005,2904,9
4,31031,4181,10
...,...,...,...
192107,28456,1281,9
192108,24458,1281,9
192109,1474,1281,3
192110,999,1281,10


In [11]:
# Verificando quantas vezes um usuário fez reviews
reviews_df["profile"].value_counts()

24591    762
25673    743
38977    648
15305    566
36106    491
        ... 
38710      1
24995      1
19297      1
18191      1
44943      1
Name: profile, Length: 47885, dtype: int64

In [12]:
# Separando apenas usuários que fizeram 10 reviews ou mais
x = reviews_df["profile"].value_counts() > 10

In [13]:
x

24591     True
25673     True
38977     True
15305     True
36106     True
         ...  
38710    False
24995    False
19297    False
18191    False
44943    False
Name: profile, Length: 47885, dtype: bool

In [14]:
y = x[x].index
print(y.shape)

(2854,)


In [15]:
# Buscando apenas as reviews de usuários que fizeram mais de 10
reviews_df = reviews_df[reviews_df["profile"].isin(y)]

In [16]:
# Mesclando reviews com animes
reviews_and_animes = reviews_df.merge(animes_df, on="anime_uid")
reviews_and_animes.head()

,profile,anime_uid,user_score,title,genre,aired,episodes,popularity,score
0,6837,34096,8,Gintama.,"['Action', 'Comedy', 'Historical', 'Parody', '...","Jan 9, 2017 to Mar 27, 2017",12.0,800,8.94
1,6837,34096,8,Gintama.,"['Action', 'Comedy', 'Historical', 'Parody', '...","Jan 9, 2017 to Mar 27, 2017",12.0,800,8.94
2,20650,34096,8,Gintama.,"['Action', 'Comedy', 'Historical', 'Parody', '...","Jan 9, 2017 to Mar 27, 2017",12.0,800,8.94
3,20650,34096,8,Gintama.,"['Action', 'Comedy', 'Historical', 'Parody', '...","Jan 9, 2017 to Mar 27, 2017",12.0,800,8.94
4,27090,34096,8,Gintama.,"['Action', 'Comedy', 'Historical', 'Parody', '...","Jan 9, 2017 to Mar 27, 2017",12.0,800,8.94


In [17]:
# Agrupando reviews e animes pela quantidade de vezes que um anime recebeu reviews
animes_reviews_times = reviews_and_animes.groupby("title")["score"].count().reset_index()

In [18]:
animes_reviews_times.rename(columns = {"score": "times_score"}, inplace=True)
animes_reviews_times

,title,times_score
0,"""0""",2
1,"""Bungaku Shoujo"" Memoire",2
2,"""Bungaku Shoujo"" Movie",12
3,"""Calpis"" Hakkou Monogatari",2
4,"""Eiji""",1
...,...,...
6535,xxxHOLiC Kei,16
6536,xxxHOLiC Movie: Manatsu no Yoru no Yume,16
6537,xxxHOLiC Rou,4
6538,ÄlDLIVE,4


In [45]:
# Mesclando reviews e animes com a quantidade de vezes que cada anime recebeu reviews
f_reviews = reviews_and_animes.merge(animes_reviews_times, on="title")

In [74]:
# Buscando animes que têm 10 ou mais reviews
final_reviews = f_reviews[f_reviews["times_score"] > 30]

In [75]:
# Removendo reviews duplicadas (mesmo usuário fez mais de uma review pro mesmo anime)
final_reviews.drop_duplicates(["profile", "title"], inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [76]:
# Transpondo a matriz pra fazer os animes se tornarem uma coluna e os perfis serem uma linha
animes_transposition = final_reviews.pivot_table(columns="profile", index="title", values="user_score")

In [78]:
# Animes que não possuem nota dos usuários passam a ter nota 0 em vez de NaN
animes_transposition.fillna(0, inplace=True)
animes_transposition.to_csv("transpo.csv")
#animes_transposition

In [64]:
from scipy.sparse import csr_matrix
# Criando uma matriz esparsa a partir dos dados tranpostos
animes_sparse = csr_matrix(animes_transposition)

In [65]:
# Treinando o algoritmo de testes
knn_model = NearestNeighbors(algorithm="brute")
knn_model.fit(animes_sparse)

NearestNeighbors(algorithm='brute')

In [66]:
# Procurando sugestões baseadas em algum anime
distances, suggestions = knn_model.kneighbors(animes_transposition.iloc[0, :].values.reshape(1, -1))

In [ ]:
animes_transposition.head()

In [67]:
for i in range(len(suggestions)):
  print(animes_transposition.index[suggestions[i]])

Index(['3-gatsu no Lion', 'Servant x Service', 'Fushigi Yuugi',
       'Owari no Seraph: Nagoya Kessen-hen', 'Flip Flappers'],
      dtype='object', name='title')
